# {{cookiecutter.project_name}} - EDA, feature engineering and feature importance

{{cookiecutter.short_description}}

A EDA cheatsheet is in https://github.com/cmawer/pycon-2017-eda-tutorial/blob/master/EDA-cheat-sheet.md

## Data Sources
- `cleaned_data.pkl`: Data cleaned in notebook 01_data_cleaning.ipynb

## Data dictionary 

The column names mean the following: 


## Changelog
- {% now 'utc', '%m-%d-%Y' %} : Started project

## Imports and definitions 

In [ ]:
# General 
from datatime import datetime
import numpy as np 
import os 
import pickle

# Plotting 
import hvplot.pandas
%matplotlib inline
import altair as alt
# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('notebook')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import holoviews as hv
hv.extension('bokeh')

# Data
import pandas as pd

# EDA 
import missingno as msno
import pandas_profiling
import shap

# ML 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
import xgboost

## Load files

In [ ]:
in_file = os.path.join('data', 'processed', 'cleaned_data.pkl')
df = pd.read_pickle(in_file)

## Feature engineering 

### Get an overview

As a first step, it is useful to follow the recommendations given by the profiler and remove columns or take some log transforms if the distributions are skewed.

In [ ]:
pandas_profiling.ProfileReport(df)

In [ ]:
hvplot.scatter_matrix(df) # using c="column name" you can add one more dimension to the plot 

Use the information here to decide which transformations you need to perform:
* log transform for skewed data
* binning might be useful for continuos variables
* simplefying categories by aggregating them 

### Deal with duplicates and missing values

Missing data can by itself give insight (http://joss.theoj.org/papers/10.21105/joss.00547) this is the reason why one might first want to visulalize it.

In [ ]:
df = df.replace('nan', np.nan)

In [ ]:
# plot nullity correlations
msno.dendrogram(df)

### Deal with outliers

Note that we have the powerful [pyod](https://github.com/yzhao062/pyod) library in the conda enviornment. You might want to check it out for more complex anonmaly or outlier detection problems. There you can also visualize and combine different approaches. 

In [ ]:
# remove outliers base on z score, everything with z >= z_score_cutoff is considered as outlier and will be dropped
z_score_cutoff = 3

z = np.abs(stats.zscore(df._get_numeric_data(), axis=1)))
df_no_outlier = df.loc[list(np.where(z < z_score_cutoff)[0])]

Possible additional steps:
* if you have categorical data, you might want to use one-hot encoding `pd.get_dummies(df)`

## Train/Test split Normalize/standardize data

In [ ]:
X = df[]
y = df[]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, train_size=0.6, random_state=0)
scaling = StandardScaler() # or use minmaxscaler or normalizer
X_train = scaling.fit_transform(X_train)
X_test = scaling.transform(X_test)

## Shap analysis of feature importance 

Shap values are a consistent way to define global and local feature importance to a model (https://christophm.github.io/interpretable-ml-book/shapley.html).
We train a gradient boost model, which often performs well, and then use Shap to explain how the model makes its decisions. 

Note that Shap values are in general a lot more consistent and valuable are the simple estimates you get from tree-based models based on how early/close to the stem a split happend. 

The shap analysis can also inform further feature engineering steps (e.g. dropping meaningless columns).

In [ ]:
parameters = {'learning_rate': [0.001, 0.01, 0.05, 0.1], 
              'max_depth': [2, 3, 4, 5, 6, 7, 8],
            }

xgb = xgboost.XGBRegressor(n_estimators=600)
xgb_grid = GridSearchCV(xgb, parameters, cv=10) # cp. arXiv:1811.12808 another method might be better in your case

xgb_grid.fit(X_train, y_train, verbose=True)

In [ ]:
explainer = shap.TreeExplainer(xgb_grid.best_estimator_)
shap_values = explainer.shap_values(X)
shap.initjs()

### Create a summary plot
* features are ordered by importance
* color coding gives the feature value
* the x-axis shows the influence on the model outcome 

In [ ]:
shap.summary_plot(shap_values, X)

### Create a interaction plot
* Shap can also be used to study if there are interactions between variables, these show up in the off-diagonals of the interaction plot


In [ ]:
interaction_values = explainer.shap_interaction_values(X)
shap.summary_plot(interaction_values, X)

### Create a force plot
* The force plot is a interesting interactive plot:
    * The x column you select is used to order the samples
    * the y values give the Shap value (influence on the model outcome)
    * the color coding shows for each data point the contributions of the different features to the model outcome (use hover)
    * the force plots are introduced in [a nature paper of the shap developer](https://www.nature.com/articles/s41551-018-0304-0) and explained [in the readme on the github site](https://github.com/slundberg/shap)

In [ ]:
display(shap.force_plot(explainer.expected_value, shap_values, X))

### Investigate single variables in detail 

Dependendence plots plot the feature value vs. the shap value of the feature. Color coded vertical dispersion is due to interaction effects.
The shap library automatically selects the feature for the color coding (the one with the highest interaction).

In case you see an interaction in a dependence plot ([a nice example is in the NHANES example notebook](https://github.com/slundberg/shap/blob/master/notebooks/tree_explainer/NHANES%20I%20Survival%20Model.ipynb)) you might want to plot the interaction values with 

```
shap.dependence_plot(
    ("X", "Color"), # select column names for x axis and color 
    shap_interaction_values, X,
    display_features=X,
)
```

i.e. a plot like 

```
shap.dependence_plot(
    ("Age", "Age"),
    shap_interaction_values, X.iloc[:2000,:],
    display_features=X_display.iloc[:2000,:]
)
```

should have now vertical dispersion since vertical dispersions are due to interactions. 

In [ ]:
shap.dependence_plot(0, shap_values, X) 